In [1]:
import pygame
import sys

# 초기화
pygame.init()

# 화면 크기 설정
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("라면 게임")

# 색상 정의
YELLOW = (255, 255, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# 한글 폰트 설정 (Windows 예시: malgun.ttf 사용)
font_path = "C:/Windows/Fonts/malgun.ttf"
small_font_path = "C:/Windows/Fonts/malgun.ttf"
font = pygame.font.Font(font_path, 40)
small_font = pygame.font.Font(small_font_path, 30)

# 버튼 클래스 정의
class Button:
    def __init__(self, x, y, width, height, text, action=None):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.action = action

    def draw(self, screen):
        pygame.draw.rect(screen, (0, 0, 255), self.rect)
        text_surface = font.render(self.text, True, WHITE)
        screen.blit(text_surface, (self.rect.x + (self.rect.width - text_surface.get_width()) // 2,
                                   self.rect.y + (self.rect.height - text_surface.get_height()) // 2))

    def is_clicked(self, pos):
        return self.rect.collidepoint(pos)

# 게임 설명 화면 표시
def show_instructions():
    screen.fill(WHITE)
    instructions_text = [
        "1. 냄비에 물을 부어야 요리를 시작할 수가 있다.",
        "물을 붓기 위해서는 주전자의 손잡이를 클릭한다.",
        "그리고 원하는 냄비의 안을 클릭해준다.",
        "2. 라면과 스프는 아무때나 넣어도 되지만 파와 계란은 꼭 물이 끓을때 넣어야 맛이 있다.",
        "물을 부을때와 마찬가지로 라면, 스프, 파, 계란을 각각 클릭한 다음",
        "원하는 냄비의 안을 클릭해준다.",
        "3. 요리가 다 되면 냄비의 아랫부분을 클릭하여 쟁반에 가져다 놓는다.",
        "4. 라면이 놓여지면 평가가 되고 라면의 맛에 따라 돈이 적립된다.",
        "그렇게 제한 시간안에 10,000원을 채우면 성공!",
        "5. 남은 냄비는 냄비가 타버렸을때 대체되는 여분이다.",
        "그러므로 없으면 대체되지 않는다.",
    ]

    y_offset = 50
    for line in instructions_text:
        text_surface = small_font.render(line, True, BLACK)
        screen.blit(text_surface, (50, y_offset))
        y_offset += 40

def start_game():
    screen.fill(WHITE)

    yellow_block = Block(YELLOW, 100, 100)
    blue_block = Block(BLUE, 300, 100)
    red_block = Block(RED, 500, 100)

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()  # 종료시 시스템을 완전히 종료시킴

            yellow_block.handle_event(event)
            blue_block.handle_event(event)
            red_block.handle_event(event)

        screen.fill(WHITE)
        yellow_block.draw(screen)
        blue_block.draw(screen)
        red_block.draw(screen)

        pygame.display.update()

def loading_screen():
    start_button = Button(300, 250, 200, 50, "Start", start_game)
    instructions_button = Button(300, 350, 200, 50, "게임 설명")
    game_button = Button(width - 220, height - 70, 200, 50, "게임하기", start_game)

    show_instructions_flag = False

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False  # 루프 종료 처리

            if event.type == pygame.MOUSEBUTTONDOWN:
                pos = event.pos
                if not show_instructions_flag:
                    if start_button.is_clicked(pos):
                        start_game()
                    elif instructions_button.is_clicked(pos):
                        show_instructions_flag = True
                else:
                    if game_button.is_clicked(pos):
                        start_game()

        screen.fill(WHITE)
        if not show_instructions_flag:
            start_button.draw(screen)
            instructions_button.draw(screen)
        else:
            show_instructions()
            game_button.draw(screen)

        pygame.display.update()

    pygame.quit()  # 프로그램 종료 시 Pygame 종료

# 시작 화면
loading_screen()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


NameError: name 'Block' is not defined